In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#create random initial spins in an NxN array?
#Need to take into account the position of the spins

#Pick one random spin and flip it
#Calculate energy difference between the old and the new state
#If E_new > E_old: keep new state
#If E_new < E_old: calculate q = p(x')/p(x).
#Draw random number: if random number lower than q: accept new state

In [3]:
def initial_spins(N):
    spin_0 = np.random.choice([1,-1])
    lattice = np.random.choice([spin_0, -spin_0], size=(N,N), p=[0.8, 0.2])
    
    return lattice



# def system_energy(lattice, coupling_constant):
#     L = len(lattice[0,:])
#     ham = 0
#     for i in range(L):
#         for j in range(L):
#             h = 0
#             h = lattice[i,j]*lattice[(i-1)%L,j-1] + lattice[i,j]*lattice[(i-1)%L,(j+1)%L] + lattice[i,j]*lattice[(i+1)%L,(j+1)%L] + lattice[i,j]*lattice[(i+1)%L,(j-1)%L]
#             ham += h
#     total_energy = -coupling_constant*ham
#     return total_energy


def system_energy(lattice, J):
#     this method is way faster the the double for loop

    energy = -J * np.sum(lattice * (np.roll(lattice, -1, axis=0) + np.roll(lattice, -1, axis=1)))
    return energy




def energy_difference(old_lattice, new_lattice):
    delta_energy = total_hamiltonian(new_lattice) - total_hamiltonian(old_lattice)
    return delta_energy


def monte_carlo(lattice, coupling_constant, temperature):
    beta = 1/temperature
    size = len(lattice[0,:])
    random_x, random_y = np.random.randint(size, size=2)
    delta_E = 2 * coupling_constant * lattice[random_x, random_y] * (lattice[(random_x-1)%size, random_y] 
                                                              + lattice[(random_x+1)%size, random_y] 
                                                              + lattice[random_x, (random_y-1)%size] 
                                                              + lattice[random_x, (random_y+1)%size])
    if delta_E <= 0:
        lattice[random_x, random_y] *= -1
        return lattice, delta_E
    if np.random.rand() < np.exp(-delta_E * beta):
        lattice[random_x, random_y] *= -1
        return lattice, delta_E
    else:
        return lattice, 0

def show_snapshot(lattice, timestep):
    
    plt.imshow(lattice, cmap='binary')
    plt.title(f'Timestep {timestep}')
    plt.show()


def ising_simulation(size, coupling_constant, temperature, timesteps):
#     two metropolis algorithm starting from the same initial lattice
    lattice = initial_spins(size)
    energies = []
    magnetizations = []
    if temperature >= 1.8 and temperature <= 2.8:
        t_to_eq = 300000
    else:
        t_to_eq = 100000
    for i in range(timesteps + t_to_eq):
        lattice, delta_E = monte_carlo(lattice, coupling_constant, temperature) 
        if i >= t_to_eq:
            energies.append(system_energy(lattice, coupling_constant)/size**2)
            magnetizations.append(np.sum(lattice)/size**2)
            
#             if i%1000 == 0:
#                 show_snapshot(lattice, i-t_to_eq)
                
    return np.array(energies), np.array(magnetizations)

def auto_correlation(magnetization, timestep):
    xi = np.zeros(timestep)
    for t in range(timestep):
        xi[t] = (np.sum(magnetization[0:timestep-t]*magnetization[t:timestep])) * (timestep-t)**(-1) -  ((np.sum(magnetization[0:timestep-t]) * (timestep-t)**(-1) ) * (np.sum(magnetization[t:timestep]) * (timestep-t)**(-1)))
        
    return xi

def corr_time(chi):
    i = 0
    integral = 0
    while chi[i] >= 0:
        integral += chi[i]/chi[0]
        i += 1
    return integral


# def average_chi(size, coupling_constant, temperature, timesteps):
#     iterations = 10
#     average_chi = np.zeros((timesteps))
#     tau_noised = np.zeros((timesteps))
#     for i in range(iterations):
#         energies, magnetizations = ising_simulation(size, coupling_constant, temperature, timesteps)
#         chi_noised = auto_correlation(magnetizations, timesteps)
#         tau_noised[i] = corr_time(chi_noised)
#         average_chi += chi_noised
#         print(i)
#     tau_std = np.std(tau_noised)
#     return average_chi/iterations, tau_std


def average_chi(size, coupling_constant, temperature, timesteps): #test
    iterations = 15
    average_chi = np.zeros((timesteps))
    tau_noised = np.zeros((iterations))
    for i in range(iterations):
        energies, magnetizations = ising_simulation(size, coupling_constant, temperature, timesteps)
        chi_noised = auto_correlation(magnetizations, timesteps)
        tau_noised[i] = corr_time(chi_noised)
        average_chi += chi_noised
    return average_chi/iterations, tau_noised

def average_tau(size, coupling_constant, temperature, timesteps):
    time_correlation, tau_noise = average_chi(size, coupling_constant, temperature, timesteps)
    average_tau = corr_time(time_correlation)
    std_tau = np.std(tau_noise)
    return average_tau, std_tau


def tau_plot(size, coupling_constant, timesteps):
    temperature = np.arange(1,4.1,0.2)
    tau_plot = np.zeros((len(temperature)))
    tau_error = np.zeros(len(temperature))
    for i in range(len(temperature)):
        tau_plot[i], tau_error[i] = average_tau(size, coupling_constant, temperature[i], timesteps)
        print(i, tau_error[i])
    return tau_plot, tau_error


def magnetic_susceptibility(magnetization, tau, temperature, n_spins, t_max):
    """Computes the magnetic susceptibility per spin for equal blocks of size 16*tau"""
    #block_length = 16*tau
    block_length = int(16*tau)
    beta = 1/temperature
    magn_susc = []    
    for j in np.arange(0, int(t_max/block_length)):
        magn_susc.append((beta/n_spins**2)*(np.mean(magnetization[j*block_length:(j+1)*block_length]**2)-np.mean(magnetization[j*block_length:(j+1)*block_length])**2))
    return np.array(magn_susc)

def specific_heat(energy, tau, temperature, n_spins, t_max):
    """Computes the specific heat per spin for equal blocks of size 16*tau"""
    block_length = int(16*tau)
    k_B = 1
    spec_heat = []
    for h in np.arange(0, np.int(t_max/block_length)):
        spec_heat.append((1/(k_B*temperature**2*n_spins**2))*(np.mean(energy[h*block_length:(h+1)*block_length]**2)-np.mean(energy[h*block_length:(h+1)*block_length])**2))
    return np.array(spec_heat)

def avg_magnetization_and_stdev(magnetization, tau, t_max):
    #t_max = sweeps
    mean_magnetization = np.mean(np.abs(magnetization)) 
    stdev_magnetization = np.sqrt((2*tau/t_max)*(np.mean(magnetization**2)-np.mean(magnetization)**2))
    return mean_magnetization, stdev_magnetization
                                  
def avg_energy_and_stdev(energy, tau, t_max):
    mean_energy = np.mean(energy)
    stdev_energy = np.sqrt((2*tau/t_max)*(np.mean(energy**2)-np.mean(energy)**2))
    return mean_energy, stdev_energy
                           
def avg_magn_susceptibility_and_stdev(magn_susceptibility, tau, t_max):
    mean_susceptibility = np.mean(magn_susceptibility)
    stdev_susceptibility = np.sqrt((2*tau/t_max)*(np.mean(magn_susceptibility**2)-np.mean(magn_susceptibility)**2))
    return mean_susceptibility, stdev_susceptibility

def avg_specific_heat_and_stdev(specific_heat, tau, t_max):
    mean_specific_heat = np.mean(specific_heat)
    stdev_specific_heat = np.sqrt((2*tau/t_max)*(np.mean(specific_heat**2)-np.mean(specific_heat)**2))
    return mean_specific_heat, stdev_specific_heat

In [4]:
#Define constants
n_spins = 50
coupling_constant = 1
temperature = 3.2
n_steps = 100000

In [5]:
# Run the simulation
# energies, magnetizations = ising_simulation(n_spins, coupling_constant, temperature, n_steps)

tau, tau_std = average_tau(n_spins, coupling_constant, temperature, n_steps)
t_max = int(20*16*tau)

In [6]:
# chi = auto_correlation(magnetizations, n_steps)
energy, magnetization = ising_simulation(n_spins, coupling_constant, temperature, t_max)

In [7]:
# # Plot the results
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
# ax1.plot(energies)
# ax1.plot(energies2)
# ax1.set_xlabel("Step")
# ax1.set_ylabel("Energy")
# ax2.plot(magnetizations)
# ax2.plot(magnetizations2)
# ax2.set_xlabel("Step")
# ax2.set_ylabel("Magnetization")
# ax2.set_ylim(-1,1)
# plt.show()

In [8]:
# plt.plot(chi/chi[0])
# plt.xlabel('Sweeps')
# plt.ylabel('Auto-correlation')
# # plt.ylim(-0.5, 1.1)

In [9]:
m_susc = magnetic_susceptibility(magnetization, tau, temperature, n_spins, t_max)

In [10]:
# for i in range(len(chi)):
#     if chi[i]<0:
#         print(chi[i], i)
#         break
print(m_susc)
print(np.mean(m_susc))
print(np.std(m_susc))

[2.03542646e-07 2.12896885e-07 3.05168117e-07 3.71502361e-07
 1.97006380e-07 2.44640304e-07 3.94897279e-07 2.83048021e-07
 4.78378218e-07 2.32646812e-07 2.54331541e-07 3.69585737e-07
 2.25642240e-07 5.32918026e-07 2.71653994e-07 1.61605489e-07
 1.03507779e-07 2.16133956e-07 2.38995076e-07 2.40915497e-07]
2.7695081781166615e-07
1.0252925162961898e-07


In [11]:
# tau = corr_time(av_correlatio)

# print(tau/n_spins**2)

In [12]:
# temp = np.arange(1,4.1,0.2)
# print(temp)

In [13]:
# plt.errorbar(temp, t_plot/n_spins**2, yerr = t_plot/(np.sqrt(10)*n_spins**2), marker = 's', linestyle = ':')
# plt.xlabel('Temperature')
# plt.ylabel('Correlation time')
# plt.savefig('tau_vs_temperature.png')

In [14]:
# print(t_error/t_plot)

In [15]:
# print(temp)